#### ***Import Libraries***

# Bloque 1: Configuración e Importación de Librerías

En este bloque se importan las principales librerías que usaremos a lo largo del notebook:
- **Numpy y Pandas:** Para manipulación de datos.
- **Matplotlib:** Para las visualizaciones.
- **PyTorch:** Para definir y entrenar el modelo de recomendación.
- **Scikit-learn:** Para la división de datos y el cálculo de métricas.

Además, se configura el dispositivo de cómputo, utilizando GPU si está disponible.

In [1]:
# Bloque 1: Configuración e Importación de Librerías

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Configurar el dispositivo: usar GPU si está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

Usando dispositivo: cuda


# Bloque 2: Carga y Preprocesamiento de Datos

En este bloque se realiza la carga del archivo `ratings.dat` y se aplica el preprocesamiento:
- Se filtran usuarios y películas con al menos 5 ratings.
- Se convierte el timestamp a formato datetime y se extraen características temporales (año, mes y día de la semana).
- Se normalizan los ratings dividiéndolos por 5 para que queden en el rango [0,1].

In [2]:
# Bloque 2: Carga y Preprocesamiento de Datos

# Cargar el dataset de ratings (asegúrate de que el archivo "ratings.dat" esté en el directorio actual o ajusta la ruta)
ratings = pd.read_csv("../data/ml-1m/ml-1m/ratings.dat", sep="::", engine="python", 
                        header=None, names=["UserID", "MovieID", "Rating", "Timestamp"])

# Mostrar las primeras filas y las dimensiones del dataset
print("Primeras 5 filas:")
print(ratings.head())
print("\nDimensiones del dataset:", ratings.shape)

# Filtrar usuarios con al menos 5 ratings
user_counts = ratings['UserID'].value_counts()
ratings = ratings[ratings['UserID'].isin(user_counts[user_counts >= 5].index)]

# Filtrar películas con al menos 5 ratings
movie_counts = ratings['MovieID'].value_counts()
ratings = ratings[ratings['MovieID'].isin(movie_counts[movie_counts >= 5].index)]

print("\nUsuarios después de filtrar:", ratings['UserID'].nunique())
print("Películas después de filtrar:", ratings['MovieID'].nunique())

# Convertir Timestamp a datetime y extraer características temporales
ratings['Datetime'] = pd.to_datetime(ratings['Timestamp'], unit='s')
ratings['Year'] = ratings['Datetime'].dt.year
ratings['Month'] = ratings['Datetime'].dt.month
ratings['DayOfWeek'] = ratings['Datetime'].dt.dayofweek

# Normalizar los ratings a [0,1]
ratings['Rating_Norm'] = ratings['Rating'] / 5.0

# Mostrar un resumen del preprocesamiento
print("\nResumen del preprocesamiento:")
print(ratings[['UserID', 'MovieID', 'Rating', 'Rating_Norm', 'Datetime', 'Year', 'Month', 'DayOfWeek']].head())

Primeras 5 filas:
   UserID  MovieID  Rating  Timestamp
0       1     1193       5  978300760
1       1      661       3  978302109
2       1      914       3  978301968
3       1     3408       4  978300275
4       1     2355       5  978824291

Dimensiones del dataset: (1000209, 4)

Usuarios después de filtrar: 6040
Películas después de filtrar: 3416

Resumen del preprocesamiento:
   UserID  MovieID  Rating  Rating_Norm            Datetime  Year  Month  \
0       1     1193       5          1.0 2000-12-31 22:12:40  2000     12   
1       1      661       3          0.6 2000-12-31 22:35:09  2000     12   
2       1      914       3          0.6 2000-12-31 22:32:48  2000     12   
3       1     3408       4          0.8 2000-12-31 22:04:35  2000     12   
4       1     2355       5          1.0 2001-01-06 23:38:11  2001      1   

   DayOfWeek  
0          6  
1          6  
2          6  
3          6  
4          5  


# Bloque 2.1: Carga y Preprocesamiento de Datos Adicionales

En este bloque cargaremos y preprocesaremos los archivos `users.dat` y `movies.dat`:
- **users.dat:** Contiene información sobre el usuario (UserID, Gender, Age, Occupation, Zip-code).  
  Se procesarán las variables:
  - *Gender*: se codifica como 0 para "F" y 1 para "M".
  - *Age* y *Occupation*: se dejan como numéricas (o se pueden transformar en categorías para generar embeddings más adelante).
  
- **movies.dat:** Contiene la información de las películas (MovieID, Title, Genres).  
  Se procesará la columna *Genres*:
  - Se separan los géneros (que vienen separados por el símbolo "|") para obtener una lista de géneros por película.
  - Se genera un mapeo de cada género a un índice para usarlo posteriormente en embeddings o en una codificación multi-hot.

In [4]:
# Bloque 2.1: Carga y Preprocesamiento de Datos Adicionales (users.dat y movies.dat)

# Cargar users.dat
users = pd.read_csv("../data/ml-1m/ml-1m/users.dat", sep="::", engine="python", header=None, 
                    names=["UserID", "Gender", "Age", "Occupation", "Zip-code"])
print("Usuarios - Primeras filas:")
print(users.head())

# Procesar users.dat
# Codificar Gender: F -> 0, M -> 1
users["Gender"] = users["Gender"].map({"F": 0, "M": 1})
# Convertir Age y Occupation a enteros (ya vienen como enteros en este dataset)
users["Age"] = users["Age"].astype(int)
users["Occupation"] = users["Occupation"].astype(int)
print("\nUsuarios procesados:")
print(users.head())

# Cargar movies.dat
movies = pd.read_csv("../data/ml-1m/ml-1m/movies.dat", sep="::", engine="python", header=None, 
                     names=["MovieID", "Title", "Genres"], encoding="latin-1")
print("\nPelículas - Primeras filas:")
print(movies.head())

# Procesar movies.dat
# Separar la columna 'Genres' en una lista de géneros para cada película
movies["Genres_list"] = movies["Genres"].apply(lambda x: x.split("|"))
print("\nPelículas con lista de géneros:")
print(movies.head())

# Crear un mapeo de género a índice
all_genres = set()
for genres in movies["Genres_list"]:
    all_genres.update(genres)
all_genres = sorted(list(all_genres))
genre_to_index = {genre: idx for idx, genre in enumerate(all_genres)}
print("\nMapa de Géneros a Índice:")
print(genre_to_index)

# Para cada película, convertir la lista de géneros a una lista de índices
movies["Genres_indices"] = movies["Genres_list"].apply(lambda gs: [genre_to_index[g] for g in gs])
print("\nPelículas con índices de géneros:")
print(movies.head())

Usuarios - Primeras filas:
   UserID Gender  Age  Occupation Zip-code
0       1      F    1          10    48067
1       2      M   56          16    70072
2       3      M   25          15    55117
3       4      M   45           7    02460
4       5      M   25          20    55455

Usuarios procesados:
   UserID  Gender  Age  Occupation Zip-code
0       1       0    1          10    48067
1       2       1   56          16    70072
2       3       1   25          15    55117
3       4       1   45           7    02460
4       5       1   25          20    55455

Películas - Primeras filas:
   MovieID                               Title                        Genres
0        1                    Toy Story (1995)   Animation|Children's|Comedy
1        2                      Jumanji (1995)  Adventure|Children's|Fantasy
2        3             Grumpier Old Men (1995)                Comedy|Romance
3        4            Waiting to Exhale (1995)                  Comedy|Drama
4        5  Fat

# Bloque 3: Fusión de Datos

En este bloque fusionamos la información de:
- **ratings.dat:** Contiene las interacciones (UserID, MovieID, Rating, Timestamp, etc.).
- **users.dat:** Información de los usuarios (Gender, Age, Occupation, Zip-code).
- **movies.dat:** Información de las películas (Title, lista de géneros y sus índices).

El resultado es un DataFrame en el que cada registro de rating incluye las características adicionales tanto del usuario como de la película.

In [5]:
# Bloque 3: Fusión de Datos

# Fusionar ratings con users utilizando 'UserID'
ratings_merged = ratings.merge(users, on="UserID", how="left")

# Fusionar el resultado con movies utilizando 'MovieID'
# Seleccionamos solo las columnas relevantes de movies
ratings_merged = ratings_merged.merge(movies[['MovieID', 'Title', 'Genres_list', 'Genres_indices']], on="MovieID", how="left")

print("Dimensiones del DataFrame fusionado:", ratings_merged.shape)
print("Primeras filas del DataFrame fusionado:")
print(ratings_merged.head())

Dimensiones del DataFrame fusionado: (999611, 16)
Primeras filas del DataFrame fusionado:
   UserID  MovieID  Rating  Timestamp            Datetime  Year  Month  \
0       1     1193       5  978300760 2000-12-31 22:12:40  2000     12   
1       1      661       3  978302109 2000-12-31 22:35:09  2000     12   
2       1      914       3  978301968 2000-12-31 22:32:48  2000     12   
3       1     3408       4  978300275 2000-12-31 22:04:35  2000     12   
4       1     2355       5  978824291 2001-01-06 23:38:11  2001      1   

   DayOfWeek  Rating_Norm  Gender  Age  Occupation Zip-code  \
0          6          1.0       0    1          10    48067   
1          6          0.6       0    1          10    48067   
2          6          0.6       0    1          10    48067   
3          6          0.8       0    1          10    48067   
4          5          1.0       0    1          10    48067   

                                    Title                       Genres_list  \
0  One 

#### 3.1.Scrapping

In [6]:
# Bloque 3.1: Integración de Datos Extra de OMDb

# Cargar el CSV extra de OMDb
omdb_df = pd.read_csv("../scrapping/omdb_movie_data.csv", encoding="utf-8")
print("Dimensiones del CSV OMDb:", omdb_df.shape)
print("Primeras filas del CSV OMDb:")
print(omdb_df.head())

# Asegurarse de que las columnas "MovieID" sean del mismo tipo en ambos DataFrames
# Convertimos a string en ambos casos
ratings_merged["MovieID"] = ratings_merged["MovieID"].astype(str)
omdb_df["MovieID"] = omdb_df["MovieID"].astype(str)

# Fusionar ratings_merged con omdb_df usando "MovieID" como llave (left join)
# Usamos el sufijo '_omdb' para las columnas del CSV extra, en caso de conflicto.
ratings_enriched = ratings_merged.merge(omdb_df, on="MovieID", how="left", suffixes=("", "_omdb"))

print("Dimensiones del DataFrame enriquecido:", ratings_enriched.shape)
print("Primeras filas del DataFrame enriquecido:")
print(ratings_enriched.head())

Dimensiones del CSV OMDb: (1438, 9)
Primeras filas del CSV OMDb:
   MovieID                                              Title  IMDbRating  \
0        1  Toy Story (1995)/The Wrestler (2008) - 99 Movi...         NaN   
1        2                                     Jumanji (1995)         NaN   
2        6                                   313: Heat (1995)         NaN   
3       18                 Four Rooms (1995) VHS Movie Review         NaN   
4       27                                Now And Then (1995)         NaN   

  Plot Director Actors      Genre Runtime      Released  
0  NaN      NaN    NaN        NaN     NaN   03 Feb 2021  
1  NaN      NaN    NaN  Talk-Show     NaN   11 May 2022  
2  NaN      NaN    NaN        NaN     NaN   26 Jul 2019  
3  NaN      NaN    NaN        NaN     NaN   24 Oct 2019  
4  NaN      NaN    NaN        NaN     NaN  30 Jun 2021   
Dimensiones del DataFrame enriquecido: (1016868, 24)
Primeras filas del DataFrame enriquecido:
   UserID MovieID  Rating  Ti

In [19]:
import numpy as np
from sklearn.model_selection import train_test_split

# Asegurarnos de que ratings_enriched tiene la columna 'Genres_multi_hot'
if 'Genres_multi_hot' not in ratings_enriched.columns:
    ratings_enriched['Genres_multi_hot'] = ratings_enriched['Genres_indices'].apply(create_multi_hot)

print("Columnas en ratings_enriched:", ratings_enriched.columns)

# Dividir el DataFrame enriquecido en train, validation y test agrupando por 'UserID'
train_list, val_list, test_list = [], [], []
for user_id, group in ratings_enriched.groupby("UserID"):
    group = group.sort_values("Timestamp")
    train, temp = train_test_split(group, test_size=0.30, random_state=42)
    val, test = train_test_split(temp, test_size=0.50, random_state=42)
    train_list.append(train)
    val_list.append(val)
    test_list.append(test)

train_df = pd.concat(train_list).reset_index(drop=True)
val_df = pd.concat(val_list).reset_index(drop=True)
test_df = pd.concat(test_list).reset_index(drop=True)

print("Tamaño de train_df:", train_df.shape)
print("Tamaño de val_df:", val_df.shape)
print("Tamaño de test_df:", test_df.shape)

Columnas en ratings_enriched: Index(['UserID', 'MovieID', 'Rating', 'Timestamp', 'Datetime', 'Year', 'Month',
       'DayOfWeek', 'Rating_Norm', 'Gender', 'Age', 'Occupation', 'Zip-code',
       'Title', 'Genres_list', 'Genres_indices', 'Title_omdb', 'IMDbRating',
       'Plot', 'Director', 'Actors', 'Genre', 'Runtime', 'Released',
       'Genres_multi_hot'],
      dtype='object')
Tamaño de train_df: (709091, 25)
Tamaño de val_df: (152350, 25)
Tamaño de test_df: (155427, 25)


# Bloque 4: Creación del Dataset Personalizado y DataLoaders (Actualizado)

En este bloque actualizaremos la clase de Dataset para incluir las nuevas características:
- Se generan índices numéricos para usuarios y películas.
- Se crea una representación multi-hot para los géneros de las películas.
- Se retorna, para cada muestra, no solo el rating y los índices, sino también:
  - `user_features`: [Gender, Age, Occupation].
  - `movie_features`: vector multi-hot de géneros.

Finalmente, se crean los DataLoaders para entrenamiento, validación y test.

In [44]:
import re
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd

class MovieLensEnhancedOMDbDataset(Dataset):
    def __init__(self, data):
        # Características básicas: índices y rating normalizado
        self.users = torch.tensor(data['userIndex'].values, dtype=torch.long)
        self.movies = torch.tensor(data['movieIndex'].values, dtype=torch.long)
        self.ratings = torch.tensor(data['Rating_Norm'].values, dtype=torch.float32)
        
        # Características del usuario: Gender, Age, Occupation
        self.user_features = torch.tensor(data[['Gender', 'Age', 'Occupation']].values, dtype=torch.float32)
        
        # Características de la película: vector multi-hot de géneros (ya generado)
        self.movie_features = data['Genres_multi_hot'].values  # Se procesa en __getitem__
        
        # Datos extra de OMDb:
        # Convertir IMDbRating a numérico; si falla se completa con 0
        self.imdb_rating = torch.tensor(
            pd.to_numeric(data['IMDbRating'], errors='coerce').fillna(0).values, 
            dtype=torch.float32
        )
        
        # Extraer y convertir la duración de Runtime usando parse_runtime
        def parse_runtime(rt):
            if pd.isna(rt):
                return 0.0
            match = re.search(r'(\d+)', str(rt))
            if match:
                return float(match.group(1))
            return 0.0
        self.runtime = torch.tensor(data['Runtime'].apply(parse_runtime).values, dtype=torch.float32)
        
        # Guardamos los campos textuales para procesamiento posterior (sin procesar aún)
        self.plot = data['Plot'].values
        self.director = data['Director'].values
        self.actors = data['Actors'].values
        self.omdb_genre = data['Genre'].values
        self.released = data['Released'].values
        
    def __len__(self):
        return len(self.ratings)
    
    def __getitem__(self, idx):
        # Convertir la lista multi-hot a tensor
        movie_feat = torch.tensor(self.movie_features[idx], dtype=torch.float32)
        return {
            "user": self.users[idx],
            "movie": self.movies[idx],
            "rating": self.ratings[idx],
            "user_features": self.user_features[idx],
            "movie_features": movie_feat,
            "imdb_rating": self.imdb_rating[idx],
            "runtime": self.runtime[idx],
            "plot": self.plot[idx],          # Texto (sin procesar)
            "director": self.director[idx],  # Texto (sin procesar)
            "actors": self.actors[idx],      # Texto (sin procesar)
            "omdb_genre": self.omdb_genre[idx],  # Texto (sin procesar)
            "released": self.released[idx]   # Texto (sin procesar)
        }

# Asegurarnos de que train_df, val_df y test_df tienen las columnas 'userIndex' y 'movieIndex'
for df in [train_df, val_df, test_df]:
    if 'userIndex' not in df.columns:
        df['userIndex'] = pd.factorize(df['UserID'])[0]
    if 'movieIndex' not in df.columns:
        df['movieIndex'] = pd.factorize(df['MovieID'])[0]

print("Columnas en train_df:", train_df.columns)

# Crear instancias del Dataset enriquecido
train_dataset_omdb = MovieLensEnhancedOMDbDataset(train_df)
val_dataset_omdb   = MovieLensEnhancedOMDbDataset(val_df)
test_dataset_omdb  = MovieLensEnhancedOMDbDataset(test_df)

from torch.utils.data._utils.collate import default_collate

def custom_collate(batch):
    collated = {}
    # Definimos explícitamente las claves que contienen texto
    text_keys = ["plot", "director", "actors", "omdb_genre", "released"]
    for key in batch[0]:
        if key in text_keys:
            collated[key] = [d[key] for d in batch]
        else:
            collated[key] = default_collate([d[key] for d in batch])
    return collated

# Crear DataLoaders
batch_size = 512
train_loader_omdb = DataLoader(train_dataset_omdb, batch_size=batch_size, shuffle=True, num_workers=0, collate_fn=custom_collate)
val_loader_omdb   = DataLoader(val_dataset_omdb, batch_size=batch_size, shuffle=False, num_workers=0, collate_fn=custom_collate)
test_loader_omdb  = DataLoader(test_dataset_omdb, batch_size=batch_size, shuffle=False, num_workers=0, collate_fn=custom_collate)

print("Tamaño del dataset de entrenamiento (con OMDb):", len(train_dataset_omdb))

Columnas en train_df: Index(['UserID', 'MovieID', 'Rating', 'Timestamp', 'Datetime', 'Year', 'Month',
       'DayOfWeek', 'Rating_Norm', 'Gender', 'Age', 'Occupation', 'Zip-code',
       'Title', 'Genres_list', 'Genres_indices', 'Title_omdb', 'IMDbRating',
       'Plot', 'Director', 'Actors', 'Genre', 'Runtime', 'Released',
       'Genres_multi_hot', 'userIndex', 'movieIndex'],
      dtype='object')
Tamaño del dataset de entrenamiento (con OMDb): 709091


# Bloque 5: Definición del Modelo Mejorado (Integrando Datos Adicionales)

En este bloque definimos un modelo que, además de utilizar los índices de usuario y película, incorpora:
- **Características del usuario:** [Gender, Age, Occupation].  
- **Características de la película:** Representación multi-hot de géneros.

La arquitectura consiste en:
1. Obtener embeddings para el ID de usuario y de película.
2. Transformar las características adicionales mediante capas lineales.
3. Combinar (concatenar) las representaciones y pasarlas por una red MLP para obtener la predicción final.

In [49]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class NeuMFEnhancedOMDb(nn.Module):
    def __init__(self, num_users, num_movies, n_genres,
                 user_embedding_dim=32, movie_embedding_dim=32,
                 user_feature_dim=3, movie_feature_output_dim=16,
                 extra_movie_feature_dim=2, extra_movie_hidden_dim=8,
                 mlp_layers=[128, 64], dropout=0.3):
        super(NeuMFEnhancedOMDb, self).__init__()
        
        # Embeddings
        self.user_embedding = nn.Embedding(num_users, user_embedding_dim)
        self.movie_embedding = nn.Embedding(num_movies, movie_embedding_dim)
        
        # Transformación de features de usuario
        self.user_feat_fc = nn.Sequential(
            nn.Linear(user_feature_dim, user_embedding_dim),
            nn.GELU()
        )
        
        # Transformación de géneros (multi-hot)
        self.movie_feat_fc = nn.Sequential(
            nn.Linear(n_genres, movie_feature_output_dim),
            nn.GELU()
        )
        
        # Transformación de IMDbRating + Runtime
        self.extra_movie_fc = nn.Sequential(
            nn.Linear(extra_movie_feature_dim, extra_movie_hidden_dim),
            nn.GELU()
        )
        
        # Dimensiones combinadas
        user_combined_dim = user_embedding_dim * 2
        movie_combined_dim = movie_embedding_dim + movie_feature_output_dim + extra_movie_hidden_dim
        fusion_input_dim = user_combined_dim + movie_combined_dim
        
        # Red MLP de fusión
        self.fusion_mlp = nn.Sequential(
            nn.Linear(fusion_input_dim, mlp_layers[0]),
            nn.BatchNorm1d(mlp_layers[0]),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(mlp_layers[0], mlp_layers[1]),
            nn.BatchNorm1d(mlp_layers[1]),
            nn.GELU(),
            nn.Dropout(dropout),
        )
        
        # Salida final normalizada (opcional: usar sigmoid y escalar a [1, 5])
        self.output = nn.Sequential(
            nn.Linear(mlp_layers[1], 1),
            nn.Sigmoid()  # Salida entre 0 y 1 → luego la escalas si quieres a [1, 5]
        )
    
    def forward(self, user_id, movie_id, user_features, movie_features, extra_movie_features):
        # Embeddings
        user_emb = self.user_embedding(user_id)
        movie_emb = self.movie_embedding(movie_id)
        
        # Procesar features
        user_extra = self.user_feat_fc(user_features)
        movie_basic = self.movie_feat_fc(movie_features)
        
        # Normalizar explícitamente IMDbRating y Runtime a [0, 1]
        x_extra = extra_movie_features.clone()
        x_extra[:, 0] = x_extra[:, 0] / 10.0        # IMDbRating
        x_extra[:, 1] = x_extra[:, 1] / 180.0       # Runtime (suponiendo máx 180 min aprox)
        extra_movie = self.extra_movie_fc(x_extra)
        
        # Combinar usuario y película
        user_vec = torch.cat([user_emb, user_extra], dim=1)
        movie_vec = torch.cat([movie_emb, movie_basic, extra_movie], dim=1)
        x = torch.cat([user_vec, movie_vec], dim=1)
        
        # Pasar por MLP
        x = self.fusion_mlp(x)
        out = self.output(x) * 4 + 1  # Reconvertimos a [1, 5]
        return out.squeeze()

# Bloque 6: Entrenamiento del Modelo Mejorado

En este bloque entrenaremos el modelo mejorado (NeuMFEnhanced) usando el dataset enriquecido (train_loader_enh, val_loader_enh y test_loader_enh).  
Se usará una función de pérdida MSE, el optimizador Adam y un scheduler para reducir la tasa de aprendizaje en caso de que la pérdida de validación no mejore.  
Implementaremos early stopping basado en la pérdida de validación para evitar sobreentrenamiento.

In [50]:
# Bloque 5: Definición del Modelo Mejorado con Datos Extra (OMDb)

# Dimensiones de entrada
num_users = train_df['userIndex'].max() + 1
num_movies = train_df['movieIndex'].max() + 1
n_genres = len(genre_to_index)  # número de géneros (dimensión del vector multi-hot)

# Instancia del modelo
model_enhanced_omdb = NeuMFEnhancedOMDb(
    num_users=num_users,
    num_movies=num_movies,
    n_genres=n_genres,
    user_embedding_dim=32,
    movie_embedding_dim=32,
    user_feature_dim=3,
    movie_feature_output_dim=16,
    extra_movie_feature_dim=2,    # IMDbRating + Runtime
    extra_movie_hidden_dim=8,
    mlp_layers=[128, 64],
    dropout=0.3
).to(device)

print(model_enhanced_omdb)


NeuMFEnhancedOMDb(
  (user_embedding): Embedding(6040, 32)
  (movie_embedding): Embedding(3416, 32)
  (user_feat_fc): Sequential(
    (0): Linear(in_features=3, out_features=32, bias=True)
    (1): GELU(approximate='none')
  )
  (movie_feat_fc): Sequential(
    (0): Linear(in_features=18, out_features=16, bias=True)
    (1): GELU(approximate='none')
  )
  (extra_movie_fc): Sequential(
    (0): Linear(in_features=2, out_features=8, bias=True)
    (1): GELU(approximate='none')
  )
  (fusion_mlp): Sequential(
    (0): Linear(in_features=120, out_features=128, bias=True)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): GELU(approximate='none')
    (3): Dropout(p=0.3, inplace=False)
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): GELU(approximate='none')
    (7): Dropout(p=0.3, inplace=False)
  )
  (output): Sequential(
    (0): Lin

In [52]:
import torch.nn.functional as F
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Función de entrenamiento
def train_enhanced_model(model, train_loader, val_loader, optimizer, criterion, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for batch in train_loader:
            user = batch['user'].to(device)
            movie = batch['movie'].to(device)
            rating = batch['rating'].to(device)
            user_features = batch['user_features'].to(device)
            movie_features = batch['movie_features'].to(device)
            
            # Extra features: IMDbRating y Runtime (dim 2)
            extra_movie_features = torch.stack([
                batch['imdb_rating'].to(device),
                batch['runtime'].to(device)
            ], dim=1)
            
            optimizer.zero_grad()
            output = model(user, movie, user_features, movie_features, extra_movie_features)
            loss = criterion(output, rating)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * rating.size(0)
        
        avg_train_loss = running_loss / len(train_loader.dataset)
        
        # Evaluación en validación
        model.eval()
        val_preds = []
        val_targets = []
        with torch.no_grad():
            for batch in val_loader:
                user = batch['user'].to(device)
                movie = batch['movie'].to(device)
                rating = batch['rating'].to(device)
                user_features = batch['user_features'].to(device)
                movie_features = batch['movie_features'].to(device)
                extra_movie_features = torch.stack([
                    batch['imdb_rating'].to(device),
                    batch['runtime'].to(device)
                ], dim=1)

                output = model(user, movie, user_features, movie_features, extra_movie_features)
                val_preds.append(output.cpu().numpy())
                val_targets.append(rating.cpu().numpy())

        val_preds = np.concatenate(val_preds)
        val_targets = np.concatenate(val_targets)
        val_rmse = np.sqrt(mean_squared_error(val_targets, val_preds))
        val_mae = mean_absolute_error(val_targets, val_preds)
        val_r2 = r2_score(val_targets, val_preds)

        print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {avg_train_loss:.4f} - "
              f"Val RMSE: {val_rmse:.4f} - MAE: {val_mae:.4f} - R²: {val_r2:.4f}")

# Entrenamiento
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model_enhanced_omdb.parameters(), lr=0.001, weight_decay=1e-5)
train_enhanced_model(model_enhanced_omdb, train_loader_omdb, val_loader_omdb, optimizer, criterion, num_epochs=10)

# Evaluación final en test
def evaluate_model_on_test(model, test_loader):
    model.eval()
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for batch in test_loader:
            user = batch['user'].to(device)
            movie = batch['movie'].to(device)
            rating = batch['rating'].to(device)
            user_features = batch['user_features'].to(device)
            movie_features = batch['movie_features'].to(device)
            extra_movie_features = torch.stack([
                batch['imdb_rating'].to(device),
                batch['runtime'].to(device)
            ], dim=1)

            output = model(user, movie, user_features, movie_features, extra_movie_features)
            all_preds.append(output.cpu().numpy())
            all_targets.append(rating.cpu().numpy())

    preds = np.concatenate(all_preds)
    targets = np.concatenate(all_targets)
    rmse = np.sqrt(mean_squared_error(targets, preds))
    mae = mean_absolute_error(targets, preds)
    r2 = r2_score(targets, preds)

    print("\nEvaluación del Modelo Mejorado (OMDb) en el conjunto de test:")
    print(f"RMSE: {rmse:.4f}")
    print(f"MAE: {mae:.4f}")
    print(f"R²: {r2:.4f}")

# Ejecutar evaluación
evaluate_model_on_test(model_enhanced_omdb, test_loader_omdb)

Epoch 1/10 - Train Loss: 0.3114 - Val RMSE: 0.3646 - MAE: 0.2881 - R²: -1.6623
Epoch 2/10 - Train Loss: 0.1335 - Val RMSE: 0.3602 - MAE: 0.2826 - R²: -1.5995
Epoch 3/10 - Train Loss: 0.1315 - Val RMSE: 0.3596 - MAE: 0.2817 - R²: -1.5897
Epoch 4/10 - Train Loss: 0.1310 - Val RMSE: 0.3594 - MAE: 0.2815 - R²: -1.5868
Epoch 5/10 - Train Loss: 0.1308 - Val RMSE: 0.3593 - MAE: 0.2814 - R²: -1.5858
Epoch 6/10 - Train Loss: 0.1308 - Val RMSE: 0.3593 - MAE: 0.2814 - R²: -1.5860
Epoch 7/10 - Train Loss: 0.1307 - Val RMSE: 0.3593 - MAE: 0.2814 - R²: -1.5861
Epoch 8/10 - Train Loss: 0.1307 - Val RMSE: 0.3593 - MAE: 0.2813 - R²: -1.5854


KeyboardInterrupt: 